In [ ]:
import random
import numpy as np 

from topology_generation import Topology
from objective_functions import DecentralizedQuadratic, DecentralizedBankNotes
from adversaries import Adversaries
from resilient_algorithms import ResilientAlgorithms
from algorithmic_framework import DistributedAlgorithmFramework
from experiments import Experiment
from plots import Plottings

In [ ]:
seed = 44
random.seed(seed)
np.random.seed(seed)
np.set_printoptions(precision=4)

### Quadratic Function Experiments (random attack)

In [ ]:
# fundamental parameters
path_name = '../results/exp_quadratic/'  # path to save the results (see experiments.py)
alg_names = ['SDMMFD', 'SDFD', 'CWTM', 'RVO']  # name of algorithms (see resilient_algorithms.py); default ['SDMMFD', 'SDFD', 'CWTM', 'RVO']
n_nodes = 40  # total number of nodes (see topology_generation.py)
n_dims = 2  # number of independent variables for quadratic functions (see objective_functions.py)
n_steps = 300  # number of time-steps in simulation (see algorithmic_framework.py); default 300
n_rounds = 4  # number of times to re-run the experiment (see experiments.py); default 5

# topology parameters
topo_type = 'r-robust'  # type of network topology (see topology_generation.py)
period = 1  # set to 1 in case of time-invariant topology (see topology_generation.py)
hyperparams_dict = {'threshold': 0.40, 'robustness': 11}  # threshold: density of additional edges; robustness: the parameter of r-robust topology (see topology_generation.py)

# adversary model and attack information
F = 2  # the parameter of F-local or F-total adversary model (see adversaries.py)
adv_model = 'local'  # model of adversaries 'local' or 'total' (see adversaries.py)
attack_info = {'type': 'random', 'param': 1.2}  # attack type and associated parameter (see adversaries.py)

# function parameters
quadratic_type = 'Q-general'  # type of generated quadratic functions 'Q-diag' or 'Q-general' (see objective_functions.py)

# framework parameters
state_inits = {'main': {'label': 'random', 'param': 20.0}, 
            'aux': {'label': 'minimizer', 'param': None}}  # states initialization (see algorithmic_framework.py)
stepsize_init = {'type': 'constant', 'param': 0.04}  # step-size initialization (see algorithmic_framework.py)
eval_info = {'n_nodes': None, 'period': None}  # using default (all) evaluation nodes and frequency (see algorithmic_framework.py)

# miscellaneous parameters
self_flag = True  # whether to consider self state as a special one which cannot be eliminated in the filtering process (see resilient_algorithms.py)
verify_flag = False  # whether to check the inputs to the algorithm every time-step (see resilient_algorithms.py)
display_flag = False  # whether to show details (for debugging purpose)


### Initialize Objects
topology = Topology(topo_type, n_nodes, period, **hyperparams_dict)
adversaries = Adversaries(topology.adjacencies[0], F, adv_model, attack_info, display_flag)
function = DecentralizedQuadratic(n_nodes, n_dims, quadratic_type)
framework = DistributedAlgorithmFramework(topology, function, adversaries, eval_info)
algorithms = [ResilientAlgorithms(alg_name, F, self_flag, verify_flag, display_flag) for alg_name in alg_names]

### Run Experiments
sim_init = {'n_rounds': n_rounds, 'n_steps': n_steps, 'state_inits': state_inits, 'stepsize_init': stepsize_init}
exp = Experiment(framework, algorithms, sim_init, path_name)

In [ ]:
# fundamental parameters
path_name = '../results/exp_quadratic/results_quadratic_240702_212529.npz'
std_factor = 1.0  # factor to multiply the standard deviation for plotting error of results (see experiments.py)

# plotting parameters (see plots.py)
common_plt_config = {'ylog': True, 'bench_label': 'min local', 'acc_flag': False, 'legend_loc': 'upper right', 'title_ext': r' ($\alpha$ = ' + f'{stepsize_init['param']})'}

dist_min_group = ['dist_min']
dist_min_plt = {'nickname': 'distance', 'ylabel': 'Distance to the Minimizer', 'limit': [None, n_steps, 0.01, 5]}
opt_gap_group = ['opt_gap']
opt_gap_plt = {'nickname': 'optimality', 'ylabel': 'Optimality Gap', 'limit': [None, n_steps, 1e-4, 50]}
cons_diam_group = ['cons_diam']
cons_diam_plt = {'nickname': 'diameter', 'ylabel': 'Regular States Diameter', 'limit': [None, min(100, n_steps), 1e-1, 10]}

### Show Results
dist_min_plt.update(common_plt_config); opt_gap_plt.update(common_plt_config); cons_diam_plt.update(common_plt_config)
metric_groups = [{'metrics': dist_min_group, 'plt': dist_min_plt}, {'metrics': opt_gap_group, 'plt': opt_gap_plt}, {'metrics': cons_diam_group, 'plt': cons_diam_plt}]
plotting = Plottings(path_name, alg_names)
plotting.process_results(metric_groups, std_factor)

### Quadratic Function Experiments (stubborn agent)

In [ ]:
# fundamental parameters
path_name = '../results/exp_quadratic_stubborn/'
alg_names = ['DGD', 'SDMMFD', 'SDFD', 'CWTM']  # ['SDMMFD', 'SDFD', 'CWTM', 'RVO']
n_nodes = 40
n_dims = 2
n_steps = 300
n_rounds = 4

# topology parameters
topo_type = 'r-robust'
period = 1
hyperparams_dict = {'threshold': 0.40, 'robustness': 11}

# adversary model and attack information
F = 2
adv_model = 'local'
attack_info = {'type': 'stubborn', 'param': (np.zeros(n_dims), 100.0)}  # perturbation attack (see adversaries.py)

# function parameters
quadratic_type = 'Q-general'

# framework parameters
state_inits = {'main': {'label': 'random', 'param': 20.0}, 
            'aux': {'label': 'minimizer', 'param': None}}
stepsize_init = {'type': 'constant', 'param': 0.02}
eval_info = {'n_nodes': None, 'period': None} 

# miscellaneous parameters
self_flag = True
verify_flag = False
display_flag = False


### Initialize Objects
topology = Topology(topo_type, n_nodes, period, **hyperparams_dict)
adversaries = Adversaries(topology.adjacencies[0], F, adv_model, attack_info, display_flag)
function = DecentralizedQuadratic(n_nodes, n_dims, quadratic_type)
framework = DistributedAlgorithmFramework(topology, function, adversaries, eval_info)
algorithms = [ResilientAlgorithms(alg_name, F, self_flag, verify_flag, display_flag) for alg_name in alg_names]
print(f"adversary indices: {adversaries.adv_indices}")

### Run Experiments
sim_init = {'n_rounds': n_rounds, 'n_steps': n_steps, 'state_inits': state_inits, 'stepsize_init': stepsize_init}
exp = Experiment(framework, algorithms, sim_init, path_name)

In [ ]:
# fundamental parameters
path_name = '../results/exp_quadratic_stubborn/results_quadratic_240703_222033.npz'
std_factor = 1.0  # factor to multiply the standard deviation for plotting error of results (see experiments.py)
stepsize = 0.04

# plotting parameters (see plots.py)
common_plt_config = {'ylog': True, 'bench_label': 'min local', 'acc_flag': False, 'legend_loc': 'upper right', 'title_ext': r' ($\alpha$ = ' + f'{stepsize})'}

dist_min_group = ['dist_min']
dist_min_plt = {'nickname': 'distance', 'ylabel': 'Distance to the Minimizer', 'limit': [None, n_steps, None, None]}
opt_gap_group = ['opt_gap']
opt_gap_plt = {'nickname': 'optimality', 'ylabel': 'Optimality Gap', 'limit': [None, n_steps, None, None]}
cons_diam_group = ['cons_diam']
cons_diam_plt = {'nickname': 'diameter', 'ylabel': 'Regular States Diameter', 'limit': [None, min(100, n_steps), None, None]}

### Show Results
dist_min_plt.update(common_plt_config); opt_gap_plt.update(common_plt_config); cons_diam_plt.update(common_plt_config)
metric_groups = [{'metrics': dist_min_group, 'plt': dist_min_plt}, {'metrics': opt_gap_group, 'plt': opt_gap_plt}, {'metrics': cons_diam_group, 'plt': cons_diam_plt}]
plotting = Plottings(path_name, alg_names)
plotting.process_results(metric_groups, std_factor)

### BankNote Experiments (random attack)

In [ ]:
# fundamental parameters
path_name = '../results/exp_banknote/'
alg_names = ['SDMMFD', 'SDFD', 'CWTM']  # default: ['SDMMFD', 'SDFD', 'CWTM']
n_nodes = 75  # default: 75
n_steps = 100  # default: 250
n_rounds = 6  # default: 6

# topology parameters
topo_type = 'r-robust'
period = 1
hyperparams_dict = {'threshold': 0.20, 'robustness': 23}

# adversary model and attack information
F = 2
adv_model = 'local'
attack_info = {'type': 'random', 'param': 1.2}  # byzantine states assignment attack (see adversaries.py)

# function parameters
bias_flag = True  # whether to distribute the data with the same label to each node (see objective_functions.py)

# framework parameters
state_inits = {'main': {'label': 'random', 'param': 20.0}, 
                'aux': {'label': 'random', 'param': 20.0}}
stepsize_init = {'type': 'constant', 'param': 2e-4}
eval_info = {'n_nodes': None, 'period': None}

# miscellaneous parameters
n_rand = None  # random state for shuffling data (see objective_functions.py)
self_flag = True
verify_flag = False
display_flag = False


### Initialize Objects
topology = Topology(topo_type, n_nodes, period, **hyperparams_dict)
adversaries = Adversaries(topology.adjacencies[0], F, adv_model, attack_info, display_flag)
function = DecentralizedBankNotes(n_nodes, bias_flag, n_rand, display_flag)
framework = DistributedAlgorithmFramework(topology, function, adversaries, eval_info)
algorithms = [ResilientAlgorithms(alg_name, F, self_flag, verify_flag, display_flag) for alg_name in alg_names]

### Run Experiments
sim_init = {'n_rounds': n_rounds, 'n_steps': n_steps, 'state_inits': state_inits, 'stepsize_init': stepsize_init}
exp = Experiment(framework, algorithms, sim_init, path_name)

In [ ]:
# fundamental parameters
path_name = '../results/exp_banknote/results_banknote_240620_211048.npz'
std_factor = 1.0

# plotting parameters  ([None, 5.0], [None, 1e3], [None, 0.6], [50, 100], [50, 100]) ([0, n_steps, 3, 6], [0, 100, 50, 3e3], [0, 100, 0.2, 3], [0, min(100, n_steps), 50, 100], [0, min(100, n_steps), 50, 100])
dist_min_group = ['dist_min']
dist_min_plt = {'nickname': 'distance', 'ylog': True, 'ylabel': 'Distance to the Minimizer', 'limit': None, 
                'acc_flag': False, 'legend_loc': 'upper right'}
opt_gap_group = ['opt_gap']
opt_gap_plt = {'nickname': 'optimality', 'ylog': True, 'ylabel': 'Optimality Gap', 'limit': None, 
                'acc_flag': False, 'legend_loc': 'upper right'}
cons_diam_group = ['cons_diam']
cons_diam_plt = {'nickname': 'diameter', 'ylog': True, 'ylabel': 'Regular States Diameter', 'limit': None, 
                    'acc_flag': False, 'legend_loc': 'upper right'}
train_acc_group = ['train_acc_avg', 'train_acc_w']
train_label_exts = ['', ' (worst)']
train_acc_plt = {'nickname':  'train', 'ylog': False, 'ylabel': 'Training Accuracy', 'limit': None, 
                    'acc_flag': True, 'legend_loc': 'lower right'}
test_acc_group = ['test_acc_avg', 'test_acc_w']
test_label_exts = ['', ' (worst)']
test_acc_plt = {'nickname': 'test', 'ylog': False, 'ylabel': 'Test Accuracy', 'limit': None,
                'acc_flag': True, 'legend_loc': 'lower right'}


### Show Results
metric_groups = [{'metrics': dist_min_group, 'plt': dist_min_plt}, {'metrics': opt_gap_group, 'plt': opt_gap_plt}, 
                 {'metrics': cons_diam_group, 'plt': cons_diam_plt}, 
                 {'metrics': train_acc_group, 'plt': train_acc_plt, 'label_exts': train_label_exts}, 
                 {'metrics': test_acc_group, 'plt': test_acc_plt, 'label_exts': test_label_exts}]
plotting = Plottings(path_name, alg_names)
plotting.process_results(metric_groups, std_factor)

### BankNote Experiments (label attack)

In [ ]:
# fundamental parameters
path_name = '../results/exp_banknote_label/'
alg_names = ['DGD', 'R-SDMMFD', 'SDFD', 'CWTM']  # default: ['DGD', 'R-SDMMFD', 'SDFD', 'CWTM']
n_nodes = 75 
n_steps = 150  # default 150
n_rounds = 6  # default 6

# topology parameters
topo_type = 'r-robust'
period = 1
hyperparams_dict = {'threshold': 0.10, 'robustness': 31}

# adversary model and attack information; default [0.50, 0.75, 1.00]
F = 10  # default 10
adv_model = 'local'
attack_info = {'type': 'label_change', 'param': 1.00}  # label flipping attack (see adversaries.py)  

# function parameters
bias_flag = False  # randomly distribute data (see objective_functions.py)

# framework parameters
state_inits = {'main': {'label': 'random', 'param': 20.0}, 
                'aux': {'label': 'random', 'param': 20.0}}
stepsize_init = {'type': 'constant', 'param': 2e-4}
eval_info = {'n_nodes': None, 'period': None}

# miscellaneous parameters
n_rand = None  # random state for shuffling data (see objective_functions.py)
self_flag = True
verify_flag = False
display_flag = False


### Initialize Objects
topology = Topology(topo_type, n_nodes, period, **hyperparams_dict)
adversaries = Adversaries(topology.adjacencies[0], F, adv_model, attack_info, display_flag)
function = DecentralizedBankNotes(n_nodes, bias_flag, n_rand, display_flag)
framework = DistributedAlgorithmFramework(topology, function, adversaries, eval_info)
algorithms = [ResilientAlgorithms(alg_name, F, self_flag, verify_flag, display_flag) for alg_name in alg_names]

### Run Experiments
sim_init = {'n_rounds': n_rounds, 'n_steps': n_steps, 'state_inits': state_inits, 'stepsize_init': stepsize_init}
exp = Experiment(framework, algorithms, sim_init, path_name)

In [ ]:
# fundamental parameters
path_name = '../results/exp_banknote_label/results_banknote_240619_181232.npz'
std_factor = None

# plotting parameters
title_extension = r' ($p$ = ' + f'{attack_info['param']})'

dist_min_group = ['dist_min']
dist_min_plt = {'nickname': 'distance', 'ylog': True, 'bench_flag': False, 'ylabel': 'Distance to the Minimizer', 
                'limit': [0, n_steps, None, None], 'acc_flag': False, 'legend_loc': 'upper right', 'title_ext': title_extension}
opt_gap_group = ['opt_gap']
opt_gap_plt = {'nickname': 'optimality', 'ylog': True, 'bench_flag': False, 'ylabel': 'Optimality Gap', 
                'limit': [0, n_steps, None, None], 'acc_flag': False, 'legend_loc': 'upper right', 'title_ext': title_extension}
cons_diam_group = ['cons_diam']
cons_diam_plt = {'nickname': 'diameter', 'ylog': True, 'bench_flag': False, 'ylabel': 'Regular States Diameter', 
                    'limit': [0, n_steps, None, None], 'acc_flag': False, 'legend_loc': 'upper right', 'title_ext': title_extension}
train_acc_group = ['train_acc_avg', 'train_acc_w']
train_label_exts = ['', ' (worst)']
train_acc_plt = {'nickname': 'train', 'ylog': False, 'ylabel': 'Train Accuracy', 'limit': [0, min(150, n_steps), 50, 100], 
                    'acc_flag': True, 'legend_loc': 'lower right', 'title_ext': title_extension}
test_acc_group = ['test_acc_avg', 'test_acc_w']
test_label_exts = ['', ' (worst)']
test_acc_plt = {'nickname': 'test', 'ylog': False, 'ylabel': 'Test Accuracy', 'limit': [0, min(150, n_steps), 50, 100], 
                    'acc_flag': True, 'legend_loc': 'lower right', 'title_ext': title_extension}


### Show Results
metric_groups = [{'metrics': dist_min_group, 'plt': dist_min_plt}, {'metrics': opt_gap_group, 'plt': opt_gap_plt}, 
                 {'metrics': cons_diam_group, 'plt': cons_diam_plt}, 
                 {'metrics': train_acc_group, 'plt': train_acc_plt, 'label_exts': train_label_exts}, 
                 {'metrics': test_acc_group, 'plt': test_acc_plt, 'label_exts': test_label_exts}]
plotting = Plottings(path_name, alg_names)
plotting.process_results(metric_groups, std_factor)

### BankNote Experiments (Gaussian attack)

In [ ]:
# fundamental parameters
path_name = '../results/exp_banknote_gaussian/'
alg_names = ['DGD', 'SDMMFD', 'SDFD', 'CWTM']  # default: ['DGD', 'SDMMFD', 'SDFD', 'CWTM']
n_nodes = 75 
n_steps = 100  # default 200
n_rounds = 6  # default 6

# topology parameters
topo_type = 'r-robust'
period = 1
hyperparams_dict = {'threshold': 0.20, 'robustness': 23}

# adversary model and attack information; default [0.02, 0.04, 0.06]
F = 2  # if F=3, then 'robustness' = 34
adv_model = 'local'
attack_info = {'type': 'perturbation', 'param': 1.000,
               'perturbation_mode': 'gaussian', 
               'broadcast_flag': True}  # perturbation attack (see adversaries.py)

# function parameters
bias_flag = False  # randomly distribute data (see objective_functions.py)

# framework parameters
state_inits = {'main': {'label': 'random', 'param': 20.0}, 
                'aux': {'label': 'random', 'param': 20.0}}
stepsize_init = {'type': 'constant', 'param': 2e-4}
eval_info = {'n_nodes': None, 'period': None}

# miscellaneous parameters
n_rand = None  # random state for shuffling data (see objective_functions.py)
self_flag = True
verify_flag = False
display_flag = False


### Initialize Objects
topology = Topology(topo_type, n_nodes, period, **hyperparams_dict)
adversaries = Adversaries(topology.adjacencies[0], F, adv_model, attack_info, display_flag)
function = DecentralizedBankNotes(n_nodes, bias_flag, n_rand, display_flag)
framework = DistributedAlgorithmFramework(topology, function, adversaries, eval_info)
algorithms = [ResilientAlgorithms(alg_name, F, self_flag, verify_flag, display_flag) for alg_name in alg_names]

### Run Experiments
sim_init = {'n_rounds': n_rounds, 'n_steps': n_steps, 'state_inits': state_inits, 'stepsize_init': stepsize_init}
exp = Experiment(framework, algorithms, sim_init, path_name)

In [ ]:
# fundamental parameters
path_name = '../results/exp_banknote_gaussian/results_banknote_240618_182811.npz'
std_factor = None

# plotting parameters
title_extension = r' ($\hat{\sigma}$ = ' + f'{attack_info['param']})'

dist_min_group = ['dist_min']
dist_min_plt = {'nickname': 'distance', 'ylog': True, 'bench_flag': False, 'ylabel': 'Distance to the Minimizer', 
                'limit': [0, n_steps, None, None], 'acc_flag': False, 'legend_loc': 'upper right', 'title_ext': title_extension}
opt_gap_group = ['opt_gap']
opt_gap_plt = {'nickname': 'optimality', 'ylog': True, 'bench_flag': False, 'ylabel': 'Optimality Gap', 
                'limit': [0, n_steps, None, None], 'acc_flag': False, 'legend_loc': 'upper right', 'title_ext': title_extension}
cons_diam_group = ['cons_diam']
cons_diam_plt = {'nickname': 'diameter', 'ylog': True, 'bench_flag': False, 'ylabel': 'Regular States Diameter', 
                 'limit': [0, n_steps, None, None], 'acc_flag': False, 'legend_loc': 'upper right', 'title_ext': title_extension}
train_acc_group = ['train_acc_avg', 'train_acc_w']
train_label_exts = ['', ' (worst)']
train_acc_plt = {'nickname': 'train', 'ylog': False, 'ylabel': 'Train Accuracy', 'limit': [0, min(100, n_steps), 50, 100], 
                 'acc_flag': True, 'legend_loc': 'lower right', 'title_ext': title_extension}
test_acc_group = ['test_acc_avg', 'test_acc_w']
test_label_exts = ['', ' (worst)']
test_acc_plt = {'nickname': 'test', 'ylog': False, 'ylabel': 'Test Accuracy', 'limit': [0, min(100, n_steps), 50, 100], 
                'acc_flag': True, 'legend_loc': 'lower right', 'title_ext': title_extension}


### Show Results
metric_groups = [{'metrics': dist_min_group, 'plt': dist_min_plt}, {'metrics': opt_gap_group, 'plt': opt_gap_plt}, 
                 {'metrics': cons_diam_group, 'plt': cons_diam_plt}, 
                 {'metrics': train_acc_group, 'plt': train_acc_plt, 'label_exts': train_label_exts}, 
                 {'metrics': test_acc_group, 'plt': test_acc_plt, 'label_exts': test_label_exts}]
plotting = Plottings(path_name, alg_names)
plotting.process_results(metric_groups, std_factor)

In [ ]:
path_name = '../results/exp_quadratic/results_quadratic_240617_220331.npz'

npzfile = np.load(path_name, allow_pickle=True)
for key in npzfile.files:
    print(key)
    print(npzfile[key])
    #print(npzfile[key] == np.zeros_like(npzfile[key]))
    print(np.array_equal(npzfile[key],np.zeros_like(npzfile[key])))
    print()
